# IMPORTS

In [28]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re
from pathlib import Path

RAW_PATH = Path("../data/raw/swiggy.csv")
CLEAN_PATH = Path("../data/processed/cleaned_data.csv")

print("Notebook Ready!")


Notebook Ready!


# DATA SOURCING AND DATA CLEANING

In [35]:
import os
from pathlib import Path

# Find project root automatically
def find_project_root(target_folder="data"):
    current = Path(os.getcwd())
    while current != current.parent:
        if (current / target_folder).exists():
            return current
        current = current.parent
    raise FileNotFoundError("Project root not found!")

ROOT = find_project_root("data")
RAW_PATH = ROOT / "data" / "raw" / "swiggy.csv"

print("Detected Project Root:", ROOT)
print("Loading:", RAW_PATH)

import pandas as pd
df = pd.read_csv(RAW_PATH)
df.head()


Detected Project Root: C:\swiggy_kmeans_project
Loading: C:\swiggy_kmeans_project\data\raw\swiggy.csv


,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [36]:
print(os.getcwd())


C:\swiggy_kmeans_project


In [37]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from pathlib import Path

# Since notebook is in project root:
RAW_PATH = Path("data/raw/swiggy.csv")

print("Loading:", RAW_PATH.resolve())

df = pd.read_csv(RAW_PATH)
df.head()


Loading: C:\swiggy_kmeans_project\data\raw\swiggy.csv


,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [38]:
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148541 entries, 0 to 148540
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            148541 non-null  int64 
 1   name          148455 non-null  object
 2   city          148541 non-null  object
 3   rating        148455 non-null  object
 4   rating_count  148455 non-null  object
 5   cost          148410 non-null  object
 6   cuisine       148442 non-null  object
 7   lic_no        148312 non-null  object
 8   link          148541 non-null  object
 9   address       148455 non-null  object
 10  menu          148541 non-null  object
dtypes: int64(1), object(10)
memory usage: 12.5+ MB


id                0
name             86
city              0
rating           86
rating_count     86
cost            131
cuisine          99
lic_no          229
link              0
address          86
menu              0
dtype: int64

In [39]:
import numpy as np

df["rating"] = df["rating"].replace("--", np.nan)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

df["rating"].fillna(df["rating"].median(), inplace=True)

df["rating_count"] = pd.to_numeric(df["rating_count"], errors="coerce").fillna(0)


In [40]:
df["cost"] = df["cost"].replace("--", np.nan)
df["cost"] = pd.to_numeric(df["cost"], errors="coerce")

# If more than 80% missing → fill by group median or global median
cost_median = df["cost"].median()
df["cost"].fillna(cost_median, inplace=True)

df["cost"].isna().sum()


np.int64(148541)

In [41]:
df["name"] = df["name"].astype(str)
df["city"] = df["city"].astype(str)
df["cuisine"] = df["cuisine"].astype(str)
df["address"] = df["address"].astype(str)


In [42]:
df["cuisine"] = df["cuisine"].str.lower().str.replace("&", ",")
df["cuisine_list"] = df["cuisine"].str.split(",")
df[["cuisine", "cuisine_list"]].head()


,cuisine,cuisine_list
0,"beverages,pizzas","[beverages, pizzas]"
1,"sweets,bakery","[sweets, bakery]"
2,beverages,[beverages]
3,"fast food,indian","[fast food, indian]"
4,"italian-american,fast food","[italian-american, fast food]"


In [43]:
df.drop_duplicates(subset=["name", "address"], inplace=True)
df.reset_index(drop=True, inplace=True)


In [44]:
df.isna().sum()

id                   0
name                 0
city                 0
rating               0
rating_count         0
cost            148402
cuisine              0
lic_no             144
link                 0
address              0
menu                 0
cuisine_list         0
dtype: int64

In [46]:
df["cost"] = pd.to_numeric(df["cost"], errors="coerce")
df["cost"].fillna(df["cost"].median(), inplace=True)


In [47]:
df.to_csv("data/processed/cleaned_data.csv", index=False)
print("Cleaned data saved.")


Cleaned data saved.
